In [1]:
# install packages if not installed yet
import sys
!{sys.executable} -m pip install lightgbm optuna

In [18]:
# imports

# data manipulation
import pandas as pd
import numpy as np

# model evaluation
from sklearn.metrics import f1_score

# lightgbm
import lightgbm as lgb
# optuna is the tuning automation tool
import optuna
import optuna.integration.lightgbm as optunalgb

In [3]:
# load data
train_file = './data/train.csv'
train_data = pd.read_csv(train_file)
train_data = pd.DataFrame(data=train_data)

test_file = './data/test.csv'
test_data = pd.read_csv(test_file)
test_data = pd.DataFrame(data=test_data)

test_ground_truths_file = './data/test_ground_truths.csv'
test_ground_truths = pd.read_csv(test_ground_truths_file)
test_ground_truths = pd.DataFrame(data=test_ground_truths)

test_data['exceeds50K'] = test_ground_truths

In [4]:
#prepare dataset
def convertCategoricalValuesToInt(df, columns):
    output = df.copy();
    
    output[columns] = output[columns].astype('category')
    cat_columns = output.select_dtypes(['category']).columns
    output[cat_columns] = output[cat_columns].apply(lambda x: x.cat.codes)
    
    return output;

train_data_lgb = train_data.drop(['native-country', 'education', 'fnlwgt'], axis=1)
test_data_lgb = test_data.drop(['native-country', 'education', 'fnlwgt'], axis=1)

x_train = train_data_lgb.drop('exceeds50K', axis=1)
y_train = train_data_lgb['exceeds50K']
x_test = test_data_lgb.drop('exceeds50K', axis=1)
y_test = test_data_lgb['exceeds50K']

features = x_train.columns
categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'sex']
categorical_features_numbered = [c for c, col in enumerate(features) if col in categorical_features]

x_train = convertCategoricalValuesToInt(x_train, categorical_features)
x_test = convertCategoricalValuesToInt(x_test, categorical_features)
print(x_train.head())

x_train = optunalgb.Dataset(x_train, label=y_train, categorical_feature=categorical_features_numbered, free_raw_data=False)
x_test = optunalgb.Dataset(x_test, label=y_test, free_raw_data=False)

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', round(f1_score(y_true, y_hat, average = 'weighted') * 100, 2), True

   age  workclass  education-num  marital-status  occupation  relationship  \
0   30          0             10               4           0             3   
1   60          4             13               0          10             1   
2   52          0             10               2           0             0   
3   37          4             13               2          12             0   
4   63          4             10               2          12             0   

   sex  capital-gain  capital-loss  hours-per-week  
0    0             0             0              30  
1    0             0             0              42  
2    1             0             0              12  
3    1             0             0              60  
4    1          7298             0              48  


In [22]:
# tune with optunalgb
parameters = {
    'metric': 'f1',
    'learning_rate': 0.001,
    'verbose': 0,
}

evals_result = {}
best_params, tuning_history = dict(), list()
model = optunalgb.train(parameters,
                       x_train,
                       valid_sets=x_test,
                       num_boost_round=5000,
                       early_stopping_rounds=1000,
                       feval=lgb_f1_score,
                       evals_result=evals_result,
                       best_params=best_params,
                       tuning_history=tuning_history,
                       verbose_eval=500)
lgb.plot_metric(evals_result, metric='f1')
print(best_params)







  0%|                                                                                            | 0/7 [00:00<?, ?it/s]





tune_feature_fraction, val_score: inf:   0%|                                                     | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 200 rounds
[500]	valid_0's f1: 86.59
[1000]	valid_0's f1: 86.8
Did not meet early stopping. Best iteration is:
[956]	valid_0's f1: 86.82








tune_feature_fraction, val_score: 86.820000:   0%|                                               | 0/7 [00:15<?, ?it/s]





tune_feature_fraction, val_score: 86.820000:  14%|#####5                                 | 1/7 [00:15<01:30, 15.16s/it][I 2020-04-26 19:58:11,708] Finished trial#0 with value: 86.82 with parameters: {'feature_fraction': 0.4}. Best is trial#0 with value: 86.82.






tune_feature_fraction, val_score: 86.820000:  14%|#####5                                 | 1/7 [00:15<01:30, 15.16s/it]

Training until validation scores don't improve for 200 rounds
[500]	valid_0's f1: 86.55
[1000]	valid_0's f1: 86.81
Did not meet early stopping. Best iteration is:
[942]	valid_0's f1: 86.81








tune_feature_fraction, val_score: 86.810000:  14%|#####5                                 | 1/7 [00:35<01:30, 15.16s/it]





tune_feature_fraction, val_score: 86.810000:  29%|###########1                           | 2/7 [00:35<01:23, 16.77s/it][I 2020-04-26 19:58:32,221] Finished trial#1 with value: 86.81 with parameters: {'feature_fraction': 0.5}. Best is trial#1 with value: 86.81.






tune_feature_fraction, val_score: 86.810000:  29%|###########1                           | 2/7 [00:35<01:23, 16.77s/it]

...







tune_num_leaves, val_score: 85.480000:  90%|######################################7    | 18/20 [04:55<00:30, 15.01s/it]





tune_num_leaves, val_score: 85.480000:  95%|########################################8  | 19/20 [04:55<00:15, 15.26s/it][I 2020-04-26 20:04:40,616] Finished trial#18 with value: 86.02 with parameters: {'num_leaves': 256}. Best is trial#1 with value: 85.48.






tune_num_leaves, val_score: 85.480000:  95%|########################################8  | 19/20 [04:55<00:15, 15.26s/it]

Training until validation scores don't improve for 200 rounds
[500]	valid_0's f1: 86.04
Early stopping, best iteration is:
[537]	valid_0's f1: 86.1








tune_num_leaves, val_score: 85.480000:  95%|########################################8  | 19/20 [05:12<00:15, 15.26s/it]





tune_num_leaves, val_score: 85.480000: 100%|###########################################| 20/20 [05:12<00:00, 15.63s/it]






  0%|                                                                                           | 0/10 [00:00<?, ?it/s]





tune_bagging_fraction_and_bagging_freq, val_score: 85.480000:   0%|                             | 0/10 [00:00<?, ?it/s]

Training until validation scores don't improve for 200 rounds
[500]	valid_0's f1: 85.1


KeyboardInterrupt: 

In [25]:
parameters = {**parameters, **best_params}
print(parameters)
evals_result = {}
model = lgb.train(parameters,
                       x_train,
                       valid_sets=x_test,
                       num_boost_round=10000,
                       early_stopping_rounds=10000,
                       feval=lgb_f1_score,
                       evals_result=evals_result,
                       verbose_eval=100)
lgb.plot_metric(evals_result, metric='f1')

{'metric': 'f1', 'learning_rate': 0.01, 'verbose': 0, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 4, 'feature_fraction': 1.0, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
Training until validation scores don't improve for 10000 rounds
[100]	valid_0's f1: 81.9
[200]	valid_0's f1: 83.49
[300]	valid_0's f1: 84.22
[400]	valid_0's f1: 84.68
[500]	valid_0's f1: 85.05
[600]	valid_0's f1: 85.17
[700]	valid_0's f1: 85.29


KeyboardInterrupt: 